Inspired from the script sisal3_extractCSVdata.py provided by Jens Fohlmeister in sisalv3 dataset [(which can be dowloaded here)](https://ora.ox.ac.uk/objects/uuid:1e91e2ac-ca9f-46e5-85f3-8d82d4d3cfd4)

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt
plt.close("all")    # close all open figures  

First, read the SISAL csv files 

In [2]:
# read all sisalv3.csv files
os.chdir("../data/sisalv3_csv")

entity_df = pd.read_csv('entity.csv')
d18O_df   = pd.read_csv("d18O.csv")
dating_df = pd.read_csv("dating.csv")

# d13C = pd.read_csv("d13C.csv")
# MgCa = pd.read_csv("Mg_Ca.csv")

# some columns of the database have a number on the first position of their name which would produce errors in python
dating_df.rename(columns = {'238U_content':'c238U_content','238U_uncertainty':'c238U_uncertainty',
    '232Th_content':'c232Th_content','c232Th_uncertainty':'c232Th_uncertainty',
    '230Th_content':'c230Th_content','c230Th_uncertainty':'c230Th_uncertainty',
    '230Th_232Th_ratio':'a230Th_232Th_ratio','230Th_232Th_ratio_uncertainty':'a230Th_232Th_ratio_uncertainty',
    '230Th_238U_activity':'a230Th_238U_activity','230Th_238U_activity_uncertainty':'a230Th_238U_activity_uncertainty',
    '234U_238U_activity':'a234U_238U_activity','234U_238U_activity_uncertainty':'a234U_238U_activity_uncertainty'},
    inplace = True)

entity_link_reference_df = pd.read_csv("entity_link_reference.csv")
original_chronology_df   = pd.read_csv("original_chronology.csv")
reference_df             = pd.read_csv("reference.csv")
sample_df                = pd.read_csv("sample.csv")
sisal_chronology_df      = pd.read_csv("sisal_chronology.csv")
site_df                  = pd.read_csv("site.csv")
os.chdir('../../notebooks')

Define time range of interest to truncate the data. 

In [3]:
# Extract required data from speleothems covering the period of interest
#   + provides all entities, which include non-14C ages and non-events  
#     during the time period

low = 0         # defines minimum age [a]
high = 500000    # defines maximum age [a] # 500'000 = U/Th datation limit, 50'000 = C14 limit

# Corrected age = corr_age = adjusted for detrital contamination
# Corrected calibrated age = c14 date_type = adjusted for dead carbon
dating_trunc = dating_df.loc[(dating_df['corr_age'] >= low) 
                         &(dating_df['corr_age'] <= high) 
                         &(dating_df['date_type'].str.find('Event')!=0)
                         ] # (dating['date_type']!='C14') &  ## Jens excluded C14 dates but for now i do not see why i should exclude them
print(f'number of samples with C14 dates is {len(dating_trunc.loc[dating_df['date_type']!='C14'])/len(dating_trunc)}')

entities_array = dating_trunc['entity_id'].to_numpy()
entities = np.unique(entities_array) 

# Remove all entities with less than 'n' dated depths
n = 3
j=0
for i in np.arange(0,len(entities)):
    entity_count = dating_trunc.entity_id[dating_trunc.entity_id==entities[i]].count()
    if entity_count < n:
        j=j+1
        dating_trunc = dating_trunc[dating_trunc.entity_id!=entities[i]].copy()
print(f'{j} entities were removed (not enough dated depths)')

entities_array = dating_trunc['entity_id'].to_numpy()
entities = np.unique(entities_array)  # provides all entities, which include >= 'n' dated depths during the required time period


number of samples with C14 dates is 0.9821724382740395
35 entities were removed (not enough dated depths)


Compute the mean d18Oc and save it in csv 

In [4]:
### define parameters (all of those will be saved in a final file)
N = len(entities)
site_id     = np.zeros(N)
site_name   = ['0']*N
rock_age    = ['0']*N
material    = ['0']*N
entity_name = ['0']*N
lon         = np.zeros(N)
lat         = np.zeros(N)
entity_id   = np.zeros(N)
mean_O      = np.zeros(N)
std_O       = np.zeros(N)
# mean_C      = np.zeros(N)
# mean_GR     = np.zeros(N)
# mean_MgCa   = np.zeros(N)
# std_C       = np.zeros(N)
# std_GR      = np.zeros(N)
# std_MgCa    = np.zeros(N)

print('number of entities : ',N)
for n in np.arange(0,N):
    plt.close("all")    # close all open figures  

    this_entity = dating_df.loc[(dating_df['entity_id'] == entities[n])]
    #######################################################################

    ### already some metadata for individual speleothems
    site_id[n]     = entity_df.site_id[(entity_df['entity_id'] == entities[n])].to_numpy()
    entity_id[n]   = entity_df.entity_id[(entity_df['entity_id'] == entities[n])].to_numpy()
    entity_name[n] = entity_df.entity_name[(entity_df['entity_id'] == entities[n])].to_list()
    site_name[n]   = site_df.site_name[(site_df['site_id'] == site_id[n])].to_list()
    lon[n]         = site_df.longitude[(site_df['site_id'] == site_id[n]).to_numpy()]
    lat[n]         = site_df.latitude[(site_df['site_id'] == site_id[n]).to_numpy()]
    
    if this_entity.material_dated.dropna().eq('calcite').all():
        material[n] = 'calcite'
    elif this_entity.material_dated.dropna().eq('aragonite').all():
        material[n] = 'aragonite'
    else:
        material[n] = 'mixed'
    print("Number:", n, entity_name[n])
    
    ### extract isotope data (d18O and d13C) and elements #####################
    ids          = sample_df.sample_id[(sample_df['entity_id']==entities[n])].to_numpy()
    age          = original_chronology_df.interp_age[original_chronology_df['sample_id'].isin(ids)].to_numpy()
    ids_with_age = original_chronology_df.sample_id[original_chronology_df['sample_id'].isin(ids)].to_numpy()
    
    d18O = d18O_df.d18O_measurement[d18O_df['sample_id'].isin(ids_with_age)].to_numpy()
    ids_with_age18 = d18O_df.sample_id[d18O_df['sample_id'].isin(ids_with_age)].to_numpy()
    age18 = original_chronology_df.interp_age[original_chronology_df['sample_id'].isin(ids_with_age18)].to_numpy()
    
    if len(ids_with_age18) != 0:
        ### determine the averages 
        if len(d18O)>0:
            mean_O[n] = np.mean(d18O[np.argwhere((age18>=low) & (age18<=high))])
            std_O[n] = np.std(d18O[np.argwhere((age18>=low) & (age18<=high))])
            
        ### define plot
        # plt.figure()
        # plt.plot(age18,d18O,'-r')
        # plt.ylabel(r"$\delta^{18}$O " + u"[\u2030 VPDB]", color='r', fontsize = 15)
        # # plt.xlim([low,high])
        # plt.tick_params(axis='y', colors='red', labelsize = 12)
        # plt.xticks([])

    #     fig,ax = plt.subplots(3,1, num='Isotopes+Elements; Entity '+str(int(entity_id[n])),figsize = (10, 7.5),clear = True)

    #     ### top plot (MgCa) ###################################################
    #     ax[0].plot(ageMgCa,MgCa_1,'-g')
    #     ax[0].set_xlim(low,high)
    #     ax[0].set_ylabel('Mg/Ca ratio [ ]', color='g', fontsize = 15)
    #     ax[0].tick_params(axis='y', colors='green', labelsize = 12)
    #     ax[0].tick_params(axis='x', labelsize = 12)
    #     ax[0].set_title(str("Entity_id: " + str(entities[n]) +" (" +
    #             entity.entity_name[(entity['entity_id'] == entities[n])].to_numpy() + ", " +
    #             str(site.site_name[(site['site_id'] == site_id[n])])[5:-31] +
    #             ")")[2:-2],fontsize = 20, x=0.5, y=1.3)
    #     ax[0].xaxis.tick_top()
    #     #######################################################################

    #     ### plot isotopes in center subplot
    #     ax[1].plot(age,d18O,'-r')
    #     if len(d13C_1)==len(age):
    #         ax3 = ax[1].twinx()
    #         ax3.plot(age13,d13C_1,'-b')
    #         ax3.tick_params(axis='y', colors='blue', labelsize = 12)
    #         ax3.set_ylabel(r"$\delta^{13}$C " + u"[\u2030 VPDB]", color='b', fontsize = 15) #use u"[\u2030]" for permil sign

    #     ax[1].set_ylabel(r"$\delta^{18}$O " + u"[\u2030 VPDB]", color='r', fontsize = 15)
    #     ax[1].set_xlim(low,high)
    #     ax[1].tick_params(axis='y', colors='red', labelsize = 12)
    #     ax[1].set_xticks([])
    #     ###################################################################
        
        
    #     ### bottom plot (growth rate after isotope depth) #################
    #     #ax[2].stairs(age,gr,'k')
    #     ax[2].stairs(gr[0:-1],age, color='black', baseline=None)
    #     ax[2].set_ylabel('growth rate \n [mm/a]', fontsize = 15)
    #     ax[2].set_xlabel('age [a BP]', fontsize = 15)
    #     ax[2].set_xlim(low, high)
    #     ax[2].tick_params(axis='both', labelsize = 12)
    #     ax[2].xaxis.set_label_coords(.5, -.2)
    #     ###################################################################

    # fig.subplots_adjust(wspace=0, hspace=0, top=0.85)
    # fig.savefig('tests/'+ str(entities[n]) +'_'+ entity.entity_name[(entity['entity_id'] == entities[n])].to_list()[0] +'.png',dpi=300)


### save file and produce an overview plot
output = pd.DataFrame({'site_id':site_id, 'site_name':site_name,
                        'longitude':lon, 'latitude':lat,
                        'entity_id':entity_id, 'eintity_name':entity_name,
                        'material':material, 
                        'mean_d18O':mean_O, 'std_d18O':std_O,
                        # 'mean_GR [mm/a]':mean_GR, 'std_GR [mm/a]':std_GR,
                        # 'mean_d13C':mean_C, 'std_d13C':std_C,
                        # 'mean_MgCa':mean_MgCa, 'std_MgCa':std_MgCa
                      })

output.replace(0, np.nan, inplace=True)
output.to_csv('../output/meanstd_values_d18O_sisal.csv')

number of entities :  811
Number: 0 ['BT-1']
Number: 1 ['BT-2.1']
Number: 2 ['BT-2.2']
Number: 3 ['BT-2.3']
Number: 4 ['BT-2.4']
Number: 5 ['BT-2.5']
Number: 6 ['BT-4']


/tmp/ipykernel_109180/1244612456.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  site_id[n]     = entity_df.site_id[(entity_df['entity_id'] == entities[n])].to_numpy()
/tmp/ipykernel_109180/1244612456.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  entity_id[n]   = entity_df.entity_id[(entity_df['entity_id'] == entities[n])].to_numpy()
/tmp/ipykernel_109180/1244612456.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lon[n]         = site_df.longitude[(site_df['site_id'] == site_id[n]).to_numpy()]
/tmp/ipykernel_109180/124461245

Number: 7 ['BT-6']
Number: 8 ['BT-8']
Number: 9 ['BT-9']
Number: 10 ['KS06-A-H']
Number: 11 ['KS06-A']
Number: 12 ['KS06-B']
Number: 13 ['KS08-1-H']
Number: 14 ['KS08-1']
Number: 15 ['KS08-2-H']
Number: 16 ['KS08-2']
Number: 17 ['KS08-6']
Number: 18 ['PAR01']
Number: 19 ['PAR03']
Number: 20 ['PAR06']
Number: 21 ['PAR07']
Number: 22 ['PAR08']
Number: 23 ['PAR16']
Number: 24 ['PAR24']
Number: 25 ['Vil-stm6']
Number: 26 ['Vil-stm9']
Number: 27 ['Vil-stm11']
Number: 28 ['Vil-stm14']
Number: 29 ['Vil-stm27']
Number: 30 ['Vil-car1']
Number: 31 ['Vil-stm1']
Number: 32 ['YK5']
Number: 33 ['YK12']
Number: 34 ['YK23']
Number: 35 ['YK61']
Number: 36 ['JFYK7']
Number: 37 ['MSD']
Number: 38 ['MSL']
Number: 39 ['PD']
Number: 40 ['YT']
Number: 41 ['T5']
Number: 42 ['T7_1999']
Number: 43 ['T7_2001']
Number: 44 ['T8']
Number: 45 ['T7_2013']
Number: 46 ['LH-70s-1']
Number: 47 ['LH-70s-2']
Number: 48 ['LH-70s-3']
Number: 49 ['TON-1']
Number: 50 ['TON-2']
Number: 51 ['JAR7']
Number: 52 ['JAR14']
Number: 5

Plot map of the mean d18Oc over the time range previously defined and save it (3D and projection)

In [5]:
out1 = output.dropna(subset='mean_d18O') # remove all 'nan' in mean d18O for plotting purposes

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lon=out1["longitude"],
    lat=out1["latitude"],
    text=out1["mean_d18O"],
    mode="markers",
    marker=dict(
        symbol="triangle-up",
        size=10,
        color=out1["mean_d18O"],
        colorscale="icefire",   # modern colormap
        cmin=-15.5,
        cmax=0,
        opacity=0.7,
        line=dict(color="white", width=1),
        colorbar=dict(
            title="δ18O (‰ VPDB)",
            ticks="outside",
            ticklen=6,
            thickness=15
        )
    )
))

fig.update_layout(
    geo=dict(
        projection=dict(type="orthographic", rotation=dict(lat=12, lon=0)),
        showland=True,
        landcolor="#f0f0f0",
        showocean=True,
        oceancolor="#def4fd",
        showcountries=False,
        showcoastlines=False,
        showframe=False
    ),
    title=dict(
        text="Global δ18O Distribution",
        x=0.5,
        xanchor="center",
        font=dict(size=20, family="Arial, sans-serif")
    ),
    margin=dict(r=20, l=20, t=50, b=20),
    template="plotly_white"
)
fig.write_html("../output/mean_values_d18O_interactive_map_sisal.html", include_plotlyjs="cdn")
fig.show()

fig.update_layout(
    geo=dict(
        projection=dict(type="natural earth"),
        showland=True,
        landcolor="#f0f0f0",
        showocean=True,
        oceancolor="#dff4fd",
        showcountries=False,
        showcoastlines=True,
        showframe=False
    )
)

fig.write_image("../output/mean_values_d18O_map_sisal.png", scale=3)